In [15]:
from utils import str2bool,evaluate_policy, Reward_adapter
from datetime import datetime
from TD3 import TD3_agent
import  gym
import numpy as np
import os, shutil
import argparse
import torch

In [16]:
'''Hyperparameter Setting'''
parser = argparse.ArgumentParser()
parser.add_argument('--dvc', type=str, default='cuda', help='running device: cuda or cpu')
parser.add_argument('--EnvIdex', type=int, default=5, help='PV1, Lch_Cv2, Humanv4, HCv4, BWv3, BWHv3')
parser.add_argument('--write', type=str2bool, default=False, help='Use SummaryWriter to record the training')
parser.add_argument('--render', type=str2bool, default=True, help='Render or Not')
parser.add_argument('--Loadmodel', type=str2bool, default=False, help='Load pretrained model or Not')
parser.add_argument('--ModelIdex', type=int, default=5000, help='which model to load')

parser.add_argument('--seed', type=int, default=0, help='random seed')
parser.add_argument('--update_every', type=int, default=50, help='training frequency')
parser.add_argument('--Max_train_steps', type=int, default=int(5e6), help='Max training steps')
parser.add_argument('--save_interval', type=int, default=int(1e5), help='Model saving interval, in steps.')
parser.add_argument('--eval_interval', type=int, default=int(2e3), help='Model evaluating interval, in steps.')

parser.add_argument('--delay_freq', type=int, default=1, help='Delayed frequency for Actor and Target Net')
parser.add_argument('--gamma', type=float, default=0.99, help='Discounted Factor')
parser.add_argument('--net_width', type=int, default=256, help='Hidden net width, s_dim-400-300-a_dim')
parser.add_argument('--a_lr', type=float, default=1e-4, help='Learning rate of actor')
parser.add_argument('--c_lr', type=float, default=1e-4, help='Learning rate of critic')
parser.add_argument('--batch_size', type=int, default=256, help='batch_size of training')
parser.add_argument('--explore_noise', type=float, default=0.15, help='exploring noise when interacting')
parser.add_argument('--explore_noise_decay', type=float, default=0.998, help='Decay rate of explore noise')
opt = parser.parse_args([])
opt.dvc = torch.device(opt.dvc) # from str to torch.device
print(opt)


Namespace(dvc=device(type='cuda'), EnvIdex=5, write=False, render=True, Loadmodel=False, ModelIdex=5000, seed=0, update_every=50, Max_train_steps=5000000, save_interval=100000, eval_interval=2000, delay_freq=1, gamma=0.99, net_width=256, a_lr=0.0001, c_lr=0.0001, batch_size=256, explore_noise=0.15, explore_noise_decay=0.998)


In [17]:
EnvName = ['Pendulum-v1','LunarLanderContinuous-v2','Humanoid-v4','HalfCheetah-v4','BipedalWalker-v3','BipedalWalkerHardcore-v3']
BrifEnvName = ['PV1', 'LLdV2', 'Humanv4', 'HCv4','BWv3', 'BWHv3']

# Build Env
env = gym.make(EnvName[opt.EnvIdex],new_step_api=True, render_mode = "human" if opt.render else None)
eval_env = gym.make(EnvName[opt.EnvIdex],new_step_api=True)
opt.state_dim = env.observation_space.shape[0]
opt.action_dim = env.action_space.shape[0]
opt.max_action = float(env.action_space.high[0])   #remark: action space【-max,max】
opt.max_e_steps = env._max_episode_steps

In [18]:
agent = TD3_agent(**vars(opt)) 

In [19]:
agent.load(BrifEnvName[opt.EnvIdex], opt.ModelIdex)

In [20]:
while True:
    s = env.reset()
    done = False
    while not done:
        # Take deterministic actions at test time
        a = agent.select_action(s, deterministic=True)
        s_next, r, dw, tr, info = env.step(a)
        done = (dw or tr)
        s = s_next

KeyboardInterrupt: 